# Assignment #2

2주차 스터디 자료에서 MC, TD TD-$\lambda$에 대해서 알 수 있었습니다.
각각 특징이 있는데 다음과 같습니다.

|         | MC    | TD    | TD-$\lambda$ |
|---------|------|------|--------------|
| update  | offline | online | online |
| target  | $G_t$ | $G_t^{(n)}$ | $G_t^\lambda $ |

## Policy Evaluation 구현하기

### Highway game 학습해보기

#### Environment

**모든 observation은 encoding된 형태로 리턴합니다!**
policy evaluation 알고리즘은 tabula 형태로 동작하고, 따라서 각 index를 observation 으로 지정하기 때문입니다.
(ex: 가위 = 1번 obs, 바위 = 2번 obs, 보 = 3번 obs)

In [1]:
import gym
import numpy as np
from IPython.display import clear_output
from stable_baselines3.common.type_aliases import GymObs, GymStepReturn

import time

class HighwayGame(gym.Env):
    def __init__(self, n_lanes: int = 3, max_steps: int = 60):
        """
        클래스의 기본적인 세팅을 초기화 합니다.
        (왠만하면) 이 클래스에서 쓰일 모든 변수는 여기서 초기화 돼야 합니다.
        Env class는 observation_space와 action_space가 필수적으로 직접 초기화 되어야 합니다.

        Parameters
        --------------------
        :param n_lanes: number of lanes for Highway game
        :param max_steps: maximum steps of environment = length of the highway
        """
        self.n_lanes = n_lanes
        self.max_steps = max_steps
        # self.road = np.load(os.path.join(PROJECT_PATH, "w1/data/road.npy"))  # 미리 준비해 둔 highway 맵
        self.road = self.generate_road()
        self.steps = 0      # 현재 steps = in-game time
        self.current_lane = n_lanes//2      # 맨 처음에 player는 lane의 중간에 위치합니다.

        self.observation_space = gym.spaces.Discrete(n_lanes)   # (0 ~ n_lane - 1) 까지의 정수
        self.action_space = gym.spaces.Discrete(n_lanes)        # (0 ~ n_lane - 1) 까지의 정수

        self.reset()    # Environment를 작동시키기 위해 초기화

    def get_obs(self) -> GymObs:
        """
        현재 관찰 가능한 상태를 리턴합니다.
        ** state와 obs는 다릅니다! obs는 agent에게 직접 제공되는 state 중 하나로,
        """
        return self.road[min(self.steps + 1, self.max_steps - 1)]

    def get_encoded_obs(self) -> GymObs:
        """
        현재 관찰 가능한 상태를 정수 형태로 인코딩 해서 리턴합니다. (only for discrete obs space)
        :return:
        """
        return np.argmin(self.get_obs())

    def reset(self) -> GymObs:
        self.road = self.generate_road()
        self.steps = 0
        self.current_lane = self.n_lanes//2
        # return self.get_obs()
        return self.get_encoded_obs()

    def step(self, action: int, debug=False) -> GymStepReturn:
        reward, done = 0, False
        info = dict()

        self.steps += 1

        if action < 0 or action >= self.n_lanes:
            reward = -1
            done = True
            info['msg'] = "Invalid action"

        elif self.road[self.steps, action]:
            reward = -1
            done = True
            info['msg'] = "You crashed!"

        elif self.steps == self.max_steps - 1:
            reward = 1
            done = True
            info['msg'] = "Finished!"

        else:
            self.current_lane = action

        if debug:
            self.render()
            print(f"Action: {action}, Reward: {reward}, Done: {done}, Info: {info}")

        return self.get_encoded_obs(), reward, done, info

    def render(self, mode="human"):
        clear_output(wait=True)
        for i, lane in enumerate(self.road[self.steps:self.steps+5]):
            repr_str = ["X" if l else " " for l in lane]
            if i == 0:
                repr_str[self.current_lane] = "O"
            print("|"+" ".join(repr_str)+"|")

    def generate_road(self) -> np.ndarray:
        """
        Highway game의 맵을 생성합니다.
        :param n_lanes: number of lanes
        :param max_steps: length of the highway
        :return: np.ndarray, shape = (max_steps, n_lanes)
        """
        road = np.ones((self.max_steps, self.n_lanes), dtype=bool)
        for i in range(self.max_steps):
            if i > 0 and road[i-1].any():
                road[i, :] = 0
            elif np.random.rand() < 0.7:
                road[i, np.random.choice(self.n_lanes)] = 0

        return road

    def play(self, policy=None):
        obs = self.reset()
        done = False
        self.render()

        while not done:
            action = policy.action(obs) if policy else self.action_space.sample()
            obs, reward, done, info = self.step(action)
            self.render()
            time.sleep(0.25)
            if done:
                if info['msg'] != "Finished!":
                    raise ValueError('제대로 학습되지 않았습니다!')
                print(info['msg'])

#### Policy Evaluation 알고리즘 준비

In [2]:
from study.w2.MC_TD import *

env = HighwayGame(n_lanes=3)
vf = ValueFunction(env.observation_space, env.action_space)
policy = GreedyPolicy(env, vf, eps=0.9)

In [4]:
total_iter = 5000

def eps_schedule(ep):
    if ep < total_iter * 0.6:
        return 0.9
    return 0.9 ** (1 + 0.02 * ep) + 0.01

Highway game에서, encoded_obs는 차가 없는 lane index를 의미합니다. (ex. obs=1 => 1번 lane이 비었음)
따라서, 학습이 제대로 됐다면 value function의 diagonal 성분 (encoded_obs == action 인 지점 => 비어있는 곳으로 움직이는 action value)가 해당 row에서 가장 높아야 합니다.

제대로 학습이 되는지 확인해 봅니다.

#### 1. MC

In [5]:
mc = MonteCarloEvaluation(policy, eps_schedule, gamma=0.9)
reward_logs = mc.evaluate(n_episodes=total_iter, steps=-1)      # MC는 episode 끝까지 step을 진행하므로 무제한 steps를 줘야 합니다.

Episode reward: -1.00	 eps: 0.010: 100%|██████████| 5000/5000 [00:02<00:00, 1839.34it/s]


In [14]:
print(f'Greedy actions: {np.argmax(policy.vf.values, axis=-1)}')
print('Value function trained by MC:')
policy.vf.values


Greedy actions: [0 1 2]
Value function trained by MC:


array([[-0.72314041, -0.89055295, -0.89663898],
       [-1.        , -0.60504666, -1.        ],
       [-1.        , -1.        , -0.6027867 ]])

#### 2. n-steps TD

In [17]:
vf = ValueFunction(env.observation_space, env.action_space)
policy = GreedyPolicy(env, vf)
td = TemporalDifferenceEvaluation(policy, eps_schedule, gamma=0.9, alpha=0.3)

In [18]:
reward_logs = td.evaluate(total_iter, steps=3)      # 3 steps TD

Episode reward: -1.00	 eps: 0.010: 100%|██████████| 5000/5000 [00:03<00:00, 1658.02it/s]


value function table을 보면, MC보다는 정확도가 떨어지는 것을 볼 수 있습니다. (diagonal 성분이 높을 수록 정확합니다.)

In [21]:
print(f'Greedy actions: {np.argmax(policy.vf.values, axis=-1)}')
print('Value function trained by 3-steps TD:')
policy.vf.values

Greedy actions: [0 1 2]
Value function trained by 3-steps TD:


array([[-0.87640209, -0.93629915, -0.94709831],
       [-1.        , -0.80743354, -1.        ],
       [-1.        , -1.        , -0.80867982]])

#### 3. TD-$\lambda$

In [26]:
vf = ValueFunction(env.observation_space, env.action_space)
policy = GreedyPolicy(env, vf)
td_lambda = TDLambda(policy, eps_schedule, gamma=0.9, lambda_=0.9, alpha=0.1)

In [27]:
reward_logs = td_lambda.evaluate(n_episodes=total_iter, steps=1)        # TD-lambda는 one-step을 사용하고도 n-steps의 효과를 냅니다.

Episode reward: -1.00	 eps: 0.010: 100%|██████████| 5000/5000 [00:03<00:00, 1553.38it/s]


In [28]:
print(f'Greedy actions: {np.argmax(policy.vf.values, axis=-1)}')
print('Value function trained by 3-steps TD-lambda:')
policy.vf.values

Greedy actions: [0 1 2]
Value function trained by 3-steps TD-lambda:


array([[-0.91223981, -0.94076518, -0.95296594],
       [-1.        , -0.82930186, -1.        ],
       [-1.        , -1.        , -0.82792356]])

## 과제 2
Monte-Carlo Evaluation 알고리즘은 구현이 되지 않아 notebook 상에서 실행되지 않습니다.
이를 구현해서 다음 env를 학습시키세요.

In [29]:
env = HighwayGame(n_lanes=5, max_steps=64)
vf = ValueFunction(env.observation_space, env.action_space)
policy = GreedyPolicy(env, vf)

In [ ]:
# MC를 이용한 학습

In [ ]:
env.play(policy)